In [9]:
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, pipeline,AutoModelForCausalLM
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer
from tqdm.notebook import tqdm
import torch
from trl.core import LengthSampler
import os
import time
from gpt import GPTConfig, GPT
from torch.nn import functional as F
import tiktoken
import json


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

## eval on awesome-chatgpt-prompts

In [3]:
dataset = load_dataset("fka/awesome-chatgpt-prompts")

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['act', 'prompt'],
        num_rows: 153
    })
})

In [5]:
print(dataset['train']['prompt'][0])

I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd


In [6]:
pretrained_model = AutoModelForCausalLM.from_pretrained('./custom_gpt2')
sft_model = AutoModelForCausalLM.from_pretrained('outputs/gpt2_sft_instruction/final_model/')
ppo_model = AutoModelForCausalLM.from_pretrained('gpt_2_ppo_model')
tokenizer = AutoTokenizer.from_pretrained('outputs/gpt2_sft_instruction/final_model/')
tokenizer.pad_token = tokenizer.eos_token


In [7]:
pretrain_pipe = pipeline(
    task='text-generation', 
    model=pretrained_model, 
    tokenizer=tokenizer, 
    max_new_tokens=512,
    device=device
)

In [7]:
sft_pipe = pipeline(
    task='text-generation', 
    model=sft_model, 
    tokenizer=tokenizer, 
    max_new_tokens=512,
    device=device
)


In [7]:
ppo_pipe = pipeline(
    task='text-generation', 
    model=ppo_model, 
    tokenizer=tokenizer, 
    max_new_tokens=512,
    device=device
)


In [8]:
template = """Human:
{}

Assistant:
"""

In [10]:
pretrain_preds = []
with open("pretrain_preds.txt", 'w') as f:
    for idx in tqdm(range(len(dataset['train']))):
        prompt = dataset['train']['prompt'][idx]
        pretrain_outputs = pretrain_pipe(
                        template.format(prompt), 
                        do_sample=True, 
                        temperature=0.7, 
                        top_k=50, 
                        top_p=0.95,
                        repetition_penalty=1.1,
                    )
        res = pretrain_outputs[0]['generated_text'].strip()
        pretrain_preds.append(res)
        f.write(res+'\n')

  0%|          | 0/153 [00:00<?, ?it/s]

In [11]:
pretrain_outs = {"pretrain_preds": pretrain_preds}

In [12]:
with open("pretrain_outs.json", 'w') as pretrain_j:
    json.dump(pretrain_outs, pretrain_j)

In [17]:
sft_preds = []
with open("sft_preds.txt", 'w') as f:
    for idx in tqdm(range(len(dataset['train']))):
        prompt = dataset['train']['prompt'][idx]
        sft_outputs = sft_pipe(
                        template.format(prompt), 
                        do_sample=True, 
                        temperature=0.7, 
                        top_k=50, 
                        top_p=0.95,
                        repetition_penalty=1.1,
                    )
        res = sft_outputs[0]['generated_text'].strip()
        sft_preds.append(res)
        f.write(res+'\n')

  0%|          | 0/153 [00:00<?, ?it/s]

In [19]:
sft_outs = {"sft_preds": sft_preds}

In [21]:
with open("sft_outs.json", 'w') as sft_j:
    json.dump(sft_outs, sft_j)

In [9]:
ppo_preds = []
with open("ppo_preds.txt", 'w') as f:
    for idx in tqdm(range(len(dataset['train']))):
        prompt = dataset['train']['prompt'][idx]
        ppo_outputs = ppo_pipe(
                        template.format(prompt), 
                        do_sample=True, 
                        temperature=0.7, 
                        top_k=50, 
                        top_p=0.95,
                        repetition_penalty=1.1,
                    )
        res = ppo_outputs[0]['generated_text'].strip()
        ppo_preds.append(res)
        f.write(res+'\n')

  0%|          | 0/153 [00:00<?, ?it/s]

In [11]:
ppo_outs = {"ppo_preds": ppo_preds}

In [12]:
with open("ppo_outs.json", 'w') as ppo_j:
    json.dump(ppo_outs, ppo_j)

### chatgpt-4o-mini judge

In [4]:
from openai import OpenAI
import os
import json
from tqdm.notebook import tqdm

In [20]:
os.environ['OPENAI_API_KEY'] = ""

In [6]:
client = OpenAI()
temperature = 0.0
model_name = "gpt-4o-mini"

In [7]:
system_prompt = '''You job is to evaluate answers from three different virtual assistant and find out which answer is more helpful, truthful and harmless to human's question. \
A better answer is usually less repetitive and it can fit well in a dialogue. If an answer look unnatural or strange in a dialogue, you shouldn't choose it. \
Human's question will start with "Human: " prefix, and virtual assistant's answer will start with "Assitant: " prefix. \
And answers from three assitants will be marked as option A, B or C. You need choose only one of the three answers, respond by either A, B, or C. Please do not answer anything other than A, B or C.'''

In [8]:
def call_gpt4o_api(input_text:str):
    response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": system_prompt}, 
                    {"role": "user", "content": input_text.strip()}
                ],
                temperature=temperature
            )
    
    try:
        return response.choices[0].message.content
    except Exception as e:
        print(e)

In [9]:
import json

In [11]:
with open("sft_outs.json") as sft:
    sft_preds = json.load(sft)['sft_preds']
with open("ppo_outs.json") as ppo:
    ppo_preds = json.load(ppo)['ppo_preds']
with open("pretrain_outs.json") as pretrain:
    pretrain_preds = json.load(pretrain)['pretrain_preds']

In [12]:
template = """A.\n{}\n\n\nB.\n{}\n\n\nC.\n{}"""

In [14]:
predictions = []
for a,b,c in zip(pretrain_preds, sft_preds, ppo_preds):
    predictions.append(call_gpt4o_api(template.format(a, b, c)))

In [17]:
predictions.count('A'), predictions.count('A')/len(predictions)

(13, 0.08496732026143791)

In [18]:
predictions.count('B'), predictions.count('B')/len(predictions)


(20, 0.13071895424836602)

In [19]:
predictions.count('C'), predictions.count('C')/len(predictions)


(120, 0.7843137254901961)

## dry run / test

In [ ]:
sft_model = AutoModelForCausalLM.from_pretrained('outputs/gpt2_sft_instruction/final_model/')
ppo_model = AutoModelForCausalLM.from_pretrained('gpt_2_ppo_model')

tokenizer = AutoTokenizer.from_pretrained('outputs/gpt2_sft_instruction/final_model/')
tokenizer.pad_token = tokenizer.eos_token


Some weights of the model checkpoint at ppo_model were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
vanilla_model = GPT(GPTConfig())
checkpoint = torch.load("model_19072.pt", map_location=torch.device('cpu'))
vanilla_model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [ ]:
enc = tiktoken.get_encoding("gpt2")

In [ ]:
ppo_pipe = pipeline(
    task='text-generation', 
    model=ppo_model, 
    tokenizer=tokenizer, 
    max_new_tokens=256,
    device=device
)


In [ ]:
sft_pipe = pipeline(
    task='text-generation', 
    model=sft_model, 
    tokenizer=tokenizer, 
    max_new_tokens=256,
    device=device
)


In [ ]:
template = """Human:
{}

Assistant:
"""


In [ ]:
inp = """Can you tell me what are the best places to visit in USA?"""

In [ ]:
vanilla_model.eval()
num_return_sequences = 3
max_length = 50
tokens = enc.encode(template.format(inp))
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to('cpu')
sample_rng = torch.Generator(device='cpu')
sample_rng.manual_seed(42)
while xgen.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        with torch.autocast(device_type='cpu', dtype=torch.bfloat16):
            logits, loss = vanilla_model(xgen) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        xgen = torch.cat((xgen, xcol), dim=1)
# print the generated text
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)
    print('---------------------------------')

Human:
Can you tell me what are the best places to visit in USA?

Assistant:
I am sorry for my child's crying and not wanting to continue my studies. At 10, years old our child's brain is developing so
---------------------------------
Human:
Can you tell me what are the best places to visit in USA?

Assistant:
Is the place perfect?
Can you tell me the best places?
If you are going in USA you can always bring me pictures of
---------------------------------
Human:
Can you tell me what are the best places to visit in USA?

Assistant:
Biological: You are very lucky!
Bios: You are so lucky! (I love trees, animals, birds)
B
---------------------------------


In [ ]:
sft_outputs = sft_pipe(
    template.format(inp), 
    do_sample=True, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    repetition_penalty=1.1,
)
print(sft_outputs[0]['generated_text'].strip())

Human:
Can you tell me what are the best places to visit in USA?

Assistant:
USA is a beautiful place. There are many cultures and religions around here, but this one has its own unique history that makes it fascinating to study as well!  I have traveled there several times already though so please check back frequently for updates on my travels!  You can find lots of information about various different ethnic groups living outside of America including their laws, customs, traditions & more, along with useful guides like Wikipedia. Also be aware that some countries do not allow photography or video recording inside houses - most likely because they don't want any privacy issues :).


In [ ]:
ppo_outputs = ppo_pipe(
    template.format(inp), 
    do_sample=True, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    repetition_penalty=1.1,
    
)
print(ppo_outputs[0]['generated_text'].strip())

Human:
Can you tell me what are the best places to visit in USA?

Assistant:
USA is one of the most popular destinations for visiting. It has a number (most) tourist attractions, such as museums, libraries and galleries, shopping malls and other public buildings, along with many good jobs opportunities ranging from hospitality companies to tech start-ups. The main attraction of USA is its extensive nature reserves which have great natural resources that offer lots more than can be bought on supermarket shelves or used by workers who work hard at their job; however, there also are some notable recreational facilities including lakeshores located around historic sites like Mount Rushmore National Park, Yosemite's Grand Canyon, Lake Tahoe and Mt. Hood. There are numerous national parks throughout the country where people enjoy hiking up hillsides off scenic mountain ranges while enjoying an abundance not found anywhere else. People come here because they want something different - fresh w

In [ ]:
inp = """Propose an idea for a self-driving car."""

In [ ]:
vanilla_model.eval()
num_return_sequences = 3
max_length = 50
tokens = enc.encode(template.format(inp))
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to('cpu')
sample_rng = torch.Generator(device='cpu')
sample_rng.manual_seed(42)
while xgen.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        with torch.autocast(device_type='cpu', dtype=torch.bfloat16):
            logits, loss = vanilla_model(xgen) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        xgen = torch.cat((xgen, xcol), dim=1)
# print the generated text
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)
    print('---------------------------------')

Human:
Propose an idea for a self-driving car.

Assistant:
- What does this example have to do with cars?
- Can you tell me of some parts of Tesla?
- What are some things that you
---------------------------------
Human:
Propose an idea for a self-driving car.

Assistant:
“We need to move from being a small startup to being the first autonomous vehicle to get a foothold in the US.”
- “
---------------------------------
Human:
Propose an idea for a self-driving car.

Assistant:
Mr. Nandakumar: "How can the government help with the process of the project"
G. S. Kumar: "How can be
---------------------------------


In [ ]:
sft_outputs = sft_pipe(
    template.format(inp), 
    do_sample=True, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    repetition_penalty=1.1,
)
print(sft_outputs[0]['generated_text'].strip())

Human:
Propose an idea for a self-driving car.

Assistant:
Imagine, in the future of technology, that there are more autonomous vehicles than human drivers and humans will ever need to drive them safely as well! Imagine how much better it would be if cars could run autonomously without steering wheel control or other devices like sensors attached to their bodies? This is what we can imagine with Autonomous Vehicles on our roads right now - they're capable even faster by automating everything from power systems, fuel delivery to navigation controls and safety cameras all over the place. I think this might sound pretty crazy but actually just something you'd want covered up at your local DMV store...

"The problem comes when people have difficulty getting around within those confines."

I agree that having autonomy may not always make sense (maybe because some users don't understand these concepts), however my opinion was based upon two things which made me feel confident enough to put t

In [ ]:
ppo_outputs = ppo_pipe(
    template.format(inp), 
    do_sample=True, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    repetition_penalty=1.1,
)
print(ppo_outputs[0]['generated_text'].strip())

Human:
Propose an idea for a self-driving car.

Assistant:
The vehicle should be able to drive itself in and out of the traffic, on its own or with assistance from other vehicles while it is being driven by another person using GPS technology. The device can also act as a safety net when driving alone—it could help avoid accidents that might occur if someone else was involved because of their surroundings (such but not limited thereto) or make decisions about where they are going based solely on what kind drivers want them to do at all times. Additionally,"the driver would have enough information available regarding which cars were traveling together so that he/she may decide whether or how to use these features." This approach works well against situations such"where there's no way around bad collisions between people who're trying very hard to keep up speed"—or even "which roads will lead you down". For example—"you don't need to worry too much anymore; just take care of yourself."" 

In [ ]:
inp = "Construct a list of tips to help reduce stress."

In [ ]:
vanilla_model.eval()
num_return_sequences = 3
max_length = 50
tokens = enc.encode(template.format(inp))
tokens = torch.tensor(tokens, dtype=torch.long)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1)
xgen = tokens.to('cpu')
sample_rng = torch.Generator(device='cpu')
sample_rng.manual_seed(42)
while xgen.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        with torch.autocast(device_type='cpu', dtype=torch.bfloat16):
            logits, loss = vanilla_model(xgen) # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1, generator=sample_rng) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        xgen = torch.cat((xgen, xcol), dim=1)
# print the generated text
for i in range(num_return_sequences):
    tokens = xgen[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(decoded)
    print('---------------------------------')

Human:
Construct a list of tips to help reduce stress.

Assistant:
- What do you like to eat?
- What do you need to do better first?
- What are you doing to get your diet right?

---------------------------------
Human:
Construct a list of tips to help reduce stress.

Assistant:
• Make time for yourself.
• Establish a routine for relaxation.
• Give the body enough time to function.
• Take a walk. Talk
---------------------------------
Human:
Construct a list of tips to help reduce stress.

Assistant:
B2K3.COM4 (8) – Self-awareness is the means
of identifying personal habits to reduce stress and reduce anxiety.
For the
---------------------------------


In [ ]:
sft_outputs = sft_pipe(
    template.format(inp), 
    do_sample=True, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    repetition_penalty=1.1,
)
print(sft_outputs[0]['generated_text'].strip())

Human:
Construct a list of tips to help reduce stress.

Assistant:
Avoid situations where you feel like it's impossible to sleep because your body is stressed and unable, or when something unexpected happens in the world that could cause problems for other people. Your best advice would be to avoid being overwhelmed by stressful events such as family vacations, holidays, etc., unless they are all about how much time needs between each event—such activities can leave feelings lasting even after these ones have ended. For example… When I had my first child (the same age), there were lots of times we stayed up late at night worrying what was going on around us instead! How important this kind-of worry really becomes isn't clear from experience; but if someone told me during pregnancy why their baby needed some extra work before he went back home? This might give them an idea of just who should spend more energy with him later than others do until his health improves further. The way forwa

In [ ]:
ppo_outputs = ppo_pipe(
    template.format(inp), 
    do_sample=True, 
    temperature=0.7, 
    top_k=50, 
    top_p=0.95,
    repetition_penalty=1.1,
)
print(ppo_outputs[0]['generated_text'].strip())

Human:
Construct a list of tips to help reduce stress.

Assistant:
1.) If you're not feeling well, consider getting some exercise in the morning and drinking lots of water at night. 2.—Avoid smoking or using alcohol when possible, as it can lead to heartburn and other symptoms. 3.-If your doctor doesn't know what's causing depression, try medicating with ayahuasca (aka kava) rather than opioids like morphine or hydromorphone. 4-Try something else—like yoga for relaxation. 5–Talk to someone about how much time they've been away from work; if there is an increase in anxiety after working hard enough hours, check back regularly so that things don' get worse before going home again. 6 —Finally! Ask them why their life has gotten better over these years. 7‑Ask yourself whether each step on this journey will have any positive effect upon others who are depressed, anxious, stressed out, irritable, angry, etc., because even though people around you may be happier while doing certain tasks inst